Getting Historical Stock Data for NYSE Equity

In [3]:
import yfinance as yf
import pandas as pd
import requests
from datetime import datetime, timedelta

# Step 1: Get the list of NYSE stocks
def get_nyse_symbols():
    url = "https://www.nasdaqtrader.com/dynamic/SymDir/nasdaqtraded.txt"
    response = requests.get(url)
    data = response.text.split("\n")

    symbols = []
    for row in data[1:]:
        cols = row.split("|")
        if len(cols) > 8 and cols[5] == "N":  # NYSE stocks are marked as "N"
            symbols.append(cols[1])

    return symbols[:-1]  # Exclude the last empty row

# Step 2: Fetch historical stock data
def fetch_stock_data(symbols, period="1mo", interval="1d"):
    all_data = []

    for symbol in symbols[:5]:  # Limit to 10 stocks for demonstration; remove [:10] for full list
        try:
            stock = yf.Ticker(symbol)
            df = stock.history(period=period, interval=interval)
            df["Symbol"] = symbol
            all_data.append(df)
        except Exception as e:
            print(f"Failed to get data for {symbol}: {e}")

    return pd.concat(all_data) if all_data else pd.DataFrame()

# Step 3: Save to CSV
def main():
    symbols = get_nyse_symbols()
    print(f"Fetched {len(symbols)} NYSE symbols.")

    stock_data = fetch_stock_data(symbols)
    if not stock_data.empty:
        stock_data.to_csv("nyse_historical_data.csv")
        print("Data saved to nyse_historical_data.csv")
    else:
        print("No data retrieved.")
main()


Fetched 7181 NYSE symbols.
Data saved to nyse_historical_data.csv


Load dummy NYSE Historical Data

In [6]:
#loading historical NYSE Data
df_nyse_history = pd.read_excel('NYSE_Historical_Data_500_Tickers.xlsx')

In [ ]:
df_nyse_history['Market Cap'] = df_nyse_history['Closing Price']*df_nyse_history['Outstanding Shares']
df_nyse_history["Market Cap Rank"] = df_nyse_history.groupby("Date")["Market Cap"].rank(method="dense", ascending=False)
df_nyse_history[df_nyse_history['Market Cap Rank'] == 1]

Load Data to Duckdb

In [58]:
#duckdb
import duckdb as duckdb
# Load Excel file into a pandas DataFrame
df_nyse_history = pd.read_excel("NYSE_Historical_Data_500_Tickers.xlsx", engine="openpyxl")
df_nyse_history['Market Cap'] = df_nyse_history['Closing Price']*df_nyse_history['Outstanding Shares']
df_nyse_history["Market Cap Rank"] = df_nyse_history.groupby("Date")["Market Cap"].rank(method="dense", ascending=False)

# Connect to DuckDB (in-memory database or specify a file)
con = duckdb.connect(database=":memory:")  # Use ":memory:" for an in-memory DB, or "your_db.duckdb" for a file

# Create table and insert data
con.execute("CREATE TABLE nyse_historical_data AS SELECT * FROM df_nyse_history")

# Verify data is loaded
print(con.execute("SELECT min(Date), max(Date) FROM nyse_historical_data LIMIT 5").fetchdf())

   min(Date)  max(Date)
0 2025-01-16 2025-04-09


**Getting top 100 stocks based on market cap for any given day**

In [ ]:

#type cast Market Cap Rank to INT32
con.execute("""ALTER TABLE nyse_historical_data ALTER "Market Cap Rank" TYPE INT32""")

#Get top 100 stocks for any input date
date_ip = input("Enter Date")
print(con.execute(f"""select * from nyse_historical_data where "Market Cap Rank" <= 100
                      and Date = '{date_ip}' """).fetchdf())

**Build Daily Index for past one month**

In [94]:
#Build Daily Index for past one month
base_value = 10
base_market_cap = 1e+12 #1 Trillion USD

daily_index_100_underlying_data = con.execute(f"""select *, sum("Market Cap") over(partition by Date) as Daily_Market_Cap
                      from nyse_historical_data where "Market Cap Rank" <= 100
                      and Date >= date_add(current_date, INTERVAL '-1 month')
                      and Date <= current_date""").fetchdf()

daily_index = con.execute(f"""select Date, (sum("Market Cap")/{base_market_cap})*{base_value} as market_index
                         from nyse_historical_data where "Market Cap Rank" <= 100
                      and Date >= date_add(current_date, INTERVAL '-1 month')
                      and Date <= current_date
                      group by 1
                      order by Date desc""").fetchdf()

**Export daily index performance and composition Data**

In [95]:
#export daily index performance and composition Data
daily_index.to_csv("Index_performance.csv", float_format="%.2f")

daily_index_100_underlying_data[['Outstanding Shares','Market Cap']] = daily_index_100_underlying_data[['Outstanding Shares','Market Cap']]/1e9
daily_index_100_underlying_data[['Daily_Market_Cap']] = daily_index_100_underlying_data[['Daily_Market_Cap']]/1e12
daily_index_100_underlying_data[['Volume']] = daily_index_100_underlying_data[['Volume']]/1e6
daily_index_100_underlying_data.rename(columns={'Outstanding Shares': 'Outstanding Shares (USD B)',
                                                'Market Cap': 'Market Cap (USD B)',
                                                'Daily_Market_Cap': 'Daily_Market_Cap (USD T)',
                                                'Volume': 'Volumne (M)'}, inplace=True)
daily_index_100_underlying_data.to_csv("Index_composition.csv", float_format="%.2f")

**Index Performance Graph**

In [96]:
import plotly.express as px

# Line Chart
fig = px.line(daily_index, x="Date", y="market_index",
              title="Market Index Over Time",
              labels={"Date": "Date", "market_index": "Market Index"},
              line_shape="linear")

# Show interactive chart
fig.show()

**Show index composition for any given date**

In [105]:
#Show index composition for any given date
import plotly.graph_objects as go
date_ip = input("Enter Date: ")
fig = go.Figure(data=[go.Table(
    header=dict(values=['Ticker'], fill_color="lightgrey", align="left"),
    cells=dict(values=[daily_index_100_underlying_data['Ticker'][daily_index_100_underlying_data['Date'] == date_ip]],
               align="left")
)])


fig.update_layout(title_text=f"Table Showing Index Composition for : {date_ip}")
fig.show()

Enter Date: 2025-02-17
